# DAMASK tutorial

- creating necessary inputs for damask
- defining the elastoplastic model(isotropic hardening) for tensile test
- runing the damask jobs

here more option is given to the user to select from damask python package itself.

Author: Yang Bai

Date  : 23.02.2022

## Importing libraries and creatign Project

In [1]:
from pyiron_continuum import Project
from damask import Rotation # this will be used in material configuration

### create a 'project' to manage all the configurations for a tensile test

In [2]:
pr = Project('isotropic')
pr.remove_jobs(silently=True) # automatically delete the existing project folder

  0%|          | 0/1 [00:00<?, ?it/s]

### Creating the Damask job

In [3]:
job = pr.create.job.DAMASK('damask_job')

In [4]:
job.elasticity(type= 'Hooke', C_11= 106.75e9,
                                   C_12= 60.41e9, C_44=28.34e9)

#### for a damask job, one needs:
- geometry information(i.e., the mesh)
- material configuration(material.yaml)
- boundary conditions(i.e., loading.yaml)

### For material configuration
#### for elastoplastic material

#### for the number of grains and grids

In [5]:
job.plasticity(type='isotropic',dot_gamma_0=0.001,
                    n=20.,xi_0=0.3e+6,
                    xi_inf=0.6e+6,a=2.,
                    h_0=1.e+6, # hardening modulus
                    M=1.,h=1.,
                    dilatation=True,output=['xi'])
# for the details of isotropic model, one is referred to https://doi.org/10.1016/j.scriptamat.2017.09.047

#### for material configuration, you need
- phase
- roation
- homogenization

In [6]:
job.phase(composition='Aluminum', lattice= 'cF',
             output_list=['F', 'P', 'F_e', 'F_p', 'L_p', 'O'])
job.rotation(Rotation.from_random, 4)

job.homogenization(method='SX', 
                 parameters={'N_constituents': 1,
                             "mechanical": {"type": "pass"}})

# now you can define your material.yaml configuration
job.set_material('Aluminum')

## For geometry information

In [7]:
job.set_grid(method="voronoi_tessellation", box_size=1.0e-5, spatial_discretization=16, num_grains=4)

## For boundary conditions (loading)

In [8]:
load_step =[{'mech_bc_dict':{'dot_F':[1e-3,0,0, 0,'x',0,  0,0,'x'],
                            'P':['x','x','x', 'x',0,'x',  'x','x',0]},
            'discretization':{'t': 10.,'N': 40},
            'additional': {'f_out': 4}
           },{'mech_bc_dict':{'dot_F':[1e-3,0,0, 0,'x',0,  0,0,'x'],
                              'P':['x','x','x', 'x',0,'x',  'x','x',0]},
            'discretization':{'t': 60.,'N': 60},
            'additional': {'f_out': 4}
           }]

solver = job.list_solvers()[0] # choose the mechanis solver
job.set_loading(solver=solver, load_steps=load_step)

In [9]:
job.run() # running your job, if you want the parallelization you can modify your 'pyiron/damask/bin/run_damask_3.0.0.sh file'

The job damask_job was saved and received the ID: 7927


2024-04-17 16:20:04,948 - pyiron_log - WARNING - Job aborted
2024-04-17 16:20:04,949 - pyiron_log - WARNING - [6db4b00e115d:06038] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)

 <<<+-  parallelization init  -+>>>

 Open MPI v4.1.6, package: Open MPI conda@a8b5c2e64a2a Distribution, ident: 4.1.6, repo rev: v4.1.6, Sep 30, 2023
 MPI standard: 3.1
 OpenMP version: 201511

 MPI processes: 1
 OMP_NUM_THREADS: 1

 <<<+-  CLI init  -+>>>

     _/_/_/      _/_/    _/      _/    _/_/      _/_/_/  _/    _/    _/_/_/
    _/    _/  _/    _/  _/_/  _/_/  _/    _/  _/        _/  _/            _/
   _/    _/  _/_/_/_/  _/  _/  _/  _/_/_/_/    _/_/    _/_/          _/_/
  _/    _/  _/    _/  _/      _/  _/    _/        _/  _/  _/            _/
 _/_/_/    _/    _/  _/      _/  _/    _/  _/_/_/    _/    _/    _/_/_/
 Grid solver

 F. Roters et al., Computational Materials Science 158:420–478, 2019
 

RuntimeError: Job aborted

In [ ]:
# plot the strain_xx vs stress_xx profile
job.plot_stress_strain(component='xx')

In [ ]:
# plot the vonMises-strain vs vonMises-stress profile
job.plot_stress_strain(von_mises=True)

In [ ]:
job.writeresults2vtk() # you can also save all the field quantities to vtk files